In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Añadir el directorio raíz del proyecto al sys.path
project_root = os.path.abspath('..')
sys.path.append(project_root)

from src.sound_signal import Signal
from src.composite_signal import CompositeSignal
from src.rhythm_signal import RhythmSignal
from src.utils import (
    calculate_note_frequencies,
    plot_spectrogram,
    plot_chromagram,
    normalize_signal
)

import random
from scipy.io.wavfile import read, write


In [ ]:
# Parámetros Generales
SAMPLE_RATE = 44100
DURATION_SEG = 5.0
AMPLITUDE = 1.0
PHASE = 0.0
OCTAVE_RANGE = (3, 5)  # Rango de octavas ajustado a frecuencias audibles
REF_FREQ = 440.0

# Tonalidades y Escalas Disponibles
TONALIDADES = ['C', 'G', 'D', 'A', 'E', 'F', 'B']
SCALAS = ['major', 'minor_natural', 'dorian', 'mixolydian', 'lydian']

# Directorios de Datos
data_dir = os.path.join(project_root, 'data')
output_dirs = {
    'tones': os.path.join(data_dir, 'tones'),
    'chords': os.path.join(data_dir, 'chords'),
    'melodies': os.path.join(data_dir, 'melodies'),
    'chord_melodies': os.path.join(data_dir, 'chord_melodies'),
    'superposed': os.path.join(data_dir, 'superposed')
}

# Crear los directorios si no existen
for dir_path in output_dirs.values():
    os.makedirs(dir_path, exist_ok=True)


In [ ]:
# Calcular las frecuencias de las notas para todas las octavas
note_freqs = calculate_note_frequencies(octave_range=OCTAVE_RANGE, ref_freq=REF_FREQ)

# Definir intervalos de escalas y modos
SCALES = {
    'major': [0, 2, 4, 5, 7, 9, 11],
    'minor_natural': [0, 2, 3, 5, 7, 8, 10],
    'dorian': [0, 2, 3, 5, 7, 9, 10],
    'mixolydian': [0, 2, 4, 5, 7, 9, 10],
    'lydian': [0, 2, 4, 6, 7, 9, 11]
}

# Mapas auxiliares para calcular las notas
SEMITONE_NOTES = {
    0: 'C', 1: 'C#', 2: 'D', 3: 'D#', 4: 'E', 5: 'F', 6: 'F#', 7: 'G',
    8: 'G#', 9: 'A', 10: 'A#', 11: 'B'
}
NOTE_SEMITONES = {v: k for k, v in SEMITONE_NOTES.items()}


In [ ]:
def get_scale_notes(tonic, scale_name, note_frequencies):
    intervals = SCALES.get(scale_name)
    if not intervals:
        print(f"Escala {scale_name} no definida.")
        return []
    
    if tonic not in NOTE_SEMITONES:
        print(f"Tónica {tonic} no válida.")
        return []
    
    tonic_semitone = NOTE_SEMITONES[tonic]
    
    # Generar las notas de la escala en todas las octavas
    scale_notes = []
    for octave in range(OCTAVE_RANGE[0], OCTAVE_RANGE[1] + 1):
        for interval in intervals:
            note_semitone = (tonic_semitone + interval) % 12
            note_name = SEMITONE_NOTES[note_semitone]
            note = f"{note_name}{octave}"
            if note in note_frequencies:
                scale_notes.append(note)
    return scale_notes

def get_note_octave(note):
    import re
    match = re.match(r'^([A-G]#?)(\d+)$', note)
    if match:
        octave = int(match.group(2))
        return octave
    else:
        return None

def generate_diatonic_chords(tonic, scale_name, note_frequencies):
    scale_notes = get_scale_notes(tonic, scale_name, note_frequencies)
    if not scale_notes:
        return {}
    
    chords = {}
    num_notes_in_scale = len(scale_notes)
    for i in range(num_notes_in_scale):
        chord_notes = []
        for j in [0, 2, 4]:  # Triada: fundamental, tercera, quinta
            note_index = (i + j) % num_notes_in_scale
            chord_notes.append(scale_notes[note_index])
        chord_name = f"{tonic}_{scale_name}_Chord_{i+1}"
        chords[chord_name] = chord_notes
    return chords


In [ ]:
def generate_and_save_tone(note, note_frequencies, output_dir):
    freq = note_frequencies.get(note)
    if freq:
        signal = Signal(amplitude=AMPLITUDE, freq=freq, phase=PHASE,
                        sample_rate=SAMPLE_RATE, duration_seg=DURATION_SEG)
        signal.generate_signal()
        filename = os.path.join(output_dir, f'{note}.wav')
        signal.save_wav(filename=filename)
        
        # Generar espectrograma y chromagram
        spectrogram_path = filename.replace('.wav', '_spectrogram.png')
        plot_spectrogram(signal.signal, signal.sample_rate, save_path=spectrogram_path)
        
        chromagram_path = filename.replace('.wav', '_chromagram.png')
        plot_chromagram(signal.signal, signal.sample_rate, save_path=chromagram_path)
        
        print(f'Tono {note} generado y guardado en {filename}')
    else:
        print(f'Frecuencia para la nota {note} no encontrada.')

def generate_and_save_chord(chord_name, notes, note_frequencies, output_dir):
    composite = CompositeSignal(sample_rate=SAMPLE_RATE, duration_seg=DURATION_SEG)
    for note in notes:
        freq = note_frequencies.get(note)
        if freq:
            composite.add_component(amplitude=AMPLITUDE, freq=freq, phase=PHASE)
        else:
            print(f'Frecuencia para la nota {note} no encontrada.')
    composite.build_signal()
    
    filename = os.path.join(output_dir, f'{chord_name}.wav')
    composite.save_wav(filename=filename)
    
    # Generar espectrograma y chromagram
    spectrogram_path = filename.replace('.wav', '_spectrogram.png')
    plot_spectrogram(composite.signal, composite.sample_rate, save_path=spectrogram_path)
    
    chromagram_path = filename.replace('.wav', '_chromagram.png')
    plot_chromagram(composite.signal, composite.sample_rate, save_path=chromagram_path)
    
    print(f'Acorde {chord_name} generado y guardado en {filename}')

def generate_and_save_melody(tonic, scale_name, note_frequencies, output_dir, num_notes=16):
    scale_notes = get_scale_notes(tonic, scale_name, note_frequencies)
    if not scale_notes:
        return
    
    # Seleccionar un rango de octavas para la melodía
    available_octaves = range(OCTAVE_RANGE[0], OCTAVE_RANGE[1] + 1)
    melody_octaves = random.sample(list(available_octaves), k=min(2, len(available_octaves)))
    melody_notes = [note for note in scale_notes if get_note_octave(note) in melody_octaves]
    
    unit_time = DURATION_SEG / num_notes
    melody = RhythmSignal(sample_rate=SAMPLE_RATE, duration_seg=DURATION_SEG, unit_time=unit_time)
    
    for i in range(num_notes):
        note = random.choice(melody_notes)
        freq = note_frequencies.get(note)
        if freq:
            segment_components = [{'amplitude': AMPLITUDE, 'freq': freq, 'phase': PHASE}]
            melody.add_segment(segment_components, label=note)
        else:
            print(f'Frecuencia para la nota {note} no encontrada.')
    
    melody.build_signal()
    
    filename = os.path.join(output_dir, f'melody_{tonic}_{scale_name}_{random.randint(0, 9999)}.wav')
    melody.save_wav(filename=filename)
    
    # Generar espectrograma y chromagram
    spectrogram_path = filename.replace('.wav', '_spectrogram.png')
    plot_spectrogram(melody.signal, melody.sample_rate, save_path=spectrogram_path)
    
    chromagram_path = filename.replace('.wav', '_chromagram.png')
    plot_chromagram(melody.signal, melody.sample_rate, save_path=chromagram_path)
    
    print(f'Melodía generada en tonalidad {tonic} {scale_name} y guardada en {filename}')

def generate_and_save_chord_progression(tonic, scale_name, note_frequencies, output_dir, num_chords=8):
    chords = generate_diatonic_chords(tonic, scale_name, note_frequencies)
    if not chords:
        return
    
    # Generar una progresión aleatoria
    chord_indices = list(range(1, len(chords) + 1))
    progression = [random.choice(chord_indices) for _ in range(num_chords)]
    
    unit_time = DURATION_SEG / num_chords
    chord_melody = RhythmSignal(sample_rate=SAMPLE_RATE, duration_seg=DURATION_SEG, unit_time=unit_time)
    
    chord_names = list(chords.keys())
    for idx in progression:
        chord_name = chord_names[idx - 1]
        chord_notes = chords[chord_name]
        segment_components = []
        for note in chord_notes:
            freq = note_frequencies.get(note)
            if freq:
                segment_components.append({'amplitude': AMPLITUDE, 'freq': freq, 'phase': PHASE})
            else:
                print(f'Frecuencia para la nota {note} no encontrada.')
        chord_melody.add_segment(segment_components, label=chord_name)
    
    chord_melody.build_signal()
    
    filename = os.path.join(output_dir, f'chord_melody_{tonic}_{scale_name}_{random.randint(0, 9999)}.wav')
    chord_melody.save_wav(filename=filename)
    
    # Generar espectrograma y chromagram
    spectrogram_path = filename.replace('.wav', '_spectrogram.png')
    plot_spectrogram(chord_melody.signal, chord_melody.sample_rate, save_path=spectrogram_path)
    
    chromagram_path = filename.replace('.wav', '_chromagram.png')
    plot_chromagram(chord_melody.signal, chord_melody.sample_rate, save_path=chromagram_path)
    
    print(f'Progresión de acordes generada en tonalidad {tonic} {scale_name} y guardada en {filename}')

def generate_and_save_superposed_signal(tonic, scale_name, output_dir):
    # Filtrar melodías y progresiones de acordes que coincidan en tonalidad y escala
    melody_files = [f for f in os.listdir(output_dirs['melodies'])
                    if f.startswith(f'melody_{tonic}_{scale_name}') and f.endswith('.wav')]
    chord_melody_files = [f for f in os.listdir(output_dirs['chord_melodies'])
                          if f.startswith(f'chord_melody_{tonic}_{scale_name}') and f.endswith('.wav')]
    
    if not melody_files or not chord_melody_files:
        print(f"No se encontraron melodías y progresiones de acordes en tonalidad {tonic} {scale_name}.")
        return
    
    # Seleccionar aleatoriamente una melodía y una progresión de acordes que coincidan
    melody_file = random.choice(melody_files)
    chord_melody_file = random.choice(chord_melody_files)
    
    melody_path = os.path.join(output_dirs['melodies'], melody_file)
    chord_melody_path = os.path.join(output_dirs['chord_melodies'], chord_melody_file)
    
    # Cargar las señales
    sample_rate_melody, melody_data = read(melody_path)
    sample_rate_chords, chords_data = read(chord_melody_path)
    
    if sample_rate_melody != sample_rate_chords:
        print("Las tasas de muestreo de la melodía y la progresión de acordes no coinciden.")
        return
    
    # Asegurar que las señales tengan la misma longitud
    min_length = min(len(melody_data), len(chords_data))
    melody_data = melody_data[:min_length]
    chords_data = chords_data[:min_length]
    
    # Convertir a float para evitar overflow
    melody_data = melody_data.astype(np.float32)
    chords_data = chords_data.astype(np.float32)
    
    # Superponer las señales y normalizar
    superposed_signal = melody_data + chords_data
    superposed_signal = normalize_signal(superposed_signal)
    
    # Convertir de vuelta a int16
    superposed_signal_int16 = (superposed_signal * 32767).astype(np.int16)
    
    # Guardar el archivo WAV superpuesto
    filename_superposed = os.path.join(
        output_dir,
        f'superposed_{tonic}_{scale_name}_{random.randint(0, 9999)}.wav'
    )
    write(filename_superposed, sample_rate_melody, superposed_signal_int16)
    
    # Generar espectrograma y chromagram
    spectrogram_path = filename_superposed.replace('.wav', '_spectrogram.png')
    plot_spectrogram(superposed_signal, sample_rate_melody, save_path=spectrogram_path)
    
    chromagram_path = filename_superposed.replace('.wav', '_chromagram.png')
    plot_chromagram(superposed_signal, sample_rate_melody, save_path=chromagram_path)
    
    print(f'Señal superpuesta generada y guardada en {filename_superposed}')


In [ ]:
# Generación de Archivos de Prueba

# Tonos Simples
print("Generando tonos simples de prueba...")
test_notes = ['A4', 'C5']
for note in test_notes:
    generate_and_save_tone(note, note_freqs, output_dirs['tones'])

# Acordes
print("\nGenerando acordes de prueba...")
test_chords = {
    'C_major': ['C4', 'E4', 'G4'],
    'G_major': ['G4', 'B4', 'D5']
}
for chord_name, notes in test_chords.items():
    generate_and_save_chord(chord_name, notes, note_freqs, output_dirs['chords'])

# Melodías
print("\nGenerando melodías de prueba...")
test_tonic = 'C'
test_scale = 'major'
for _ in range(2):
    generate_and_save_melody(test_tonic, test_scale, note_freqs, output_dirs['melodies'], num_notes=8)

# Progresiones de Acordes
print("\nGenerando progresiones de acordes de prueba...")
for _ in range(2):
    generate_and_save_chord_progression(test_tonic, test_scale, note_freqs, output_dirs['chord_melodies'], num_chords=4)

# Señales Superpuestas
print("\nGenerando señales superpuestas de prueba...")
for _ in range(2):
    generate_and_save_superposed_signal(test_tonic, test_scale, output_dirs['superposed'])


In [ ]:
def generate_dataset(note_frequencies):
    """
    Genera tonos, acordes, melodías, progresiones de acordes y señales superpuestas para el conjunto de datos.
    """
    # Generar tonos
    print("Generando tonos simples...")
    for note in note_frequencies.keys():
        generate_and_save_tone(note, note_frequencies, output_dirs['tones'])
    
    # Generar acordes diatónicos
    print("\nGenerando acordes diatónicos...")
    for tonic in TONALIDADES:
        for scale_name in ['major', 'minor_natural']:
            chords = generate_diatonic_chords(tonic, scale_name, note_frequencies)
            for chord_name, notes in chords.items():
                generate_and_save_chord(chord_name, notes, note_frequencies, output_dirs['chords'])
    
    # Generar melodías
    print("\nGenerando melodías...")
    for tonic in TONALIDADES:
        for scale_name in SCALAS:
            for _ in range(5):  # Generar 5 melodías por tonalidad y escala
                generate_and_save_melody(tonic, scale_name, note_frequencies, output_dirs['melodies'])
    
    # Generar progresiones de acordes
    print("\nGenerando progresiones de acordes...")
    for tonic in TONALIDADES:
        for scale_name in ['major', 'minor_natural']:
            for _ in range(5):  # Generar 5 progresiones por tonalidad y escala
                generate_and_save_chord_progression(tonic, scale_name, note_frequencies, output_dirs['chord_melodies'])
    
    # Generar señales superpuestas
    print("\nGenerando señales superpuestas...")
    for tonic in TONALIDADES:
        for scale_name in ['major', 'minor_natural']:
            for _ in range(5):  # Generar 5 señales superpuestas por tonalidad y escala
                generate_and_save_superposed_signal(tonic, scale_name, output_dirs['superposed'])


In [ ]:
# Generar el dataset completo
generate_dataset(note_freqs)
